# Exportacion oficial Transformers.js (Qwen2 fine-tuned)

Este cuaderno exporta el modelo merged a ONNX en el **formato oficial de Transformers.js** y luego genera la cuantizacion **q4f16**.

In [ ]:
# Instalar dependencias (Colab)
!pip -q uninstall -y optimum diffusers peft
!pip -q install "optimum[exporters]==1.23.3" "transformers==4.45.2" onnx onnxruntime onnxslim onnxscript

In [ ]:
# Montar Google Drive
from google.colab import drive

drive.mount('/content/drive')

## Copiar el modelo merged desde Drive
Ajusta la ruta si tu zip esta en otra carpeta de Drive.

In [ ]:
import os

MERGED_ZIP = '/content/drive/MyDrive/fine-tuning/epicrisis-merged.zip'
!cp "$MERGED_ZIP" /content/epicrisis-merged.zip
!unzip -q /content/epicrisis-merged.zip -d /content

MODEL_DIR = '/content/epicrisis-merged'
if not os.path.isdir(MODEL_DIR):
    alt = '/content/epicrisis-merged/epicrisis-merged'
    if os.path.isdir(alt):
        MODEL_DIR = alt

print('MODEL_DIR:', MODEL_DIR)
!ls -la "$MODEL_DIR" | head -n 20

## Clonar Transformers.js (scripts oficiales)

In [ ]:
!rm -rf /content/transformers.js
!git clone --depth 1 https://github.com/huggingface/transformers.js /content/transformers.js

## Exportar y cuantizar a q4f16
Esto genera el layout oficial (archivos ONNX en `onnx/`).

In [ ]:
%cd /content

!python /content/transformers.js/scripts/convert.py   --model_id epicrisis-merged   --task text-generation-with-past   --device cuda   --opset 18   --skip_validation   --skip_onnxslim   --quantize   --modes q4f16   --output_parent_dir /content/exported

## Renombrar salida y actualizar config.json

In [ ]:
import json
import os
import shutil

src = '/content/exported/epicrisis-merged'
dst = '/content/epicrisis-q4f16-finetuned-tjs'

if os.path.exists(dst):
    shutil.rmtree(dst)
shutil.move(src, dst)

onnx_dir = os.path.join(dst, 'onnx')
use_ext = {}
for f in os.listdir(onnx_dir):
    if f.endswith('.onnx'):
        data_path = os.path.join(onnx_dir, f + '_data')
        if os.path.exists(data_path):
            use_ext[f] = 1

cfg_path = os.path.join(dst, 'config.json')
with open(cfg_path, 'r') as f:
    cfg = json.load(f)

cfg.setdefault('transformers.js_config', {})
cfg['transformers.js_config']['dtype'] = 'q4f16'
cfg['transformers.js_config']['kv_cache_dtype'] = {
    'q4f16': 'float16',
    'fp16': 'float16',
}
if use_ext:
    cfg['transformers.js_config']['use_external_data_format'] = use_ext

with open(cfg_path, 'w') as f:
    json.dump(cfg, f, indent=2)

print('Salida:', dst)
!ls -la "$dst" | head -n 20
!ls -la "$onnx_dir" | head -n 40

## Zip y copiar a Drive

In [ ]:
!zip -r /content/epicrisis-q4f16-finetuned-tjs.zip /content/epicrisis-q4f16-finetuned-tjs
!cp /content/epicrisis-q4f16-finetuned-tjs.zip /content/drive/MyDrive/fine-tuning/